In [83]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time

#list api keys here
from config import api_key

state = 'Oaklahoma'
state_code = 40

In [35]:
url = 'http://api.census.gov/data/2010/sf1?get=P0010001,NAME&for=county:*&in=state:40&key=' + api_key
response = requests.get(url).json()
county_name = []
county_code = []
census_variable_code = []
counter = 0;
for county in response:
    if (counter > 0):
        county_name.append(county[1])
        county_code.append(county[3])
        census_variable_code.append(county[0])
    counter = counter + 1
    
all_counties = {"County Name" : county_name,
               "County Code" : county_code,
               "Census Code" : census_variable_code}

all_counties_df = pd.DataFrame(all_counties)
all_counties_df.head()

,County Name,County Code,Census Code
0,Adair County,001,22683
1,Alfalfa County,003,5642
2,Atoka County,005,14182
3,Beaver County,007,5636
4,Beckham County,009,22119


In [40]:
url = 'http://api.census.gov/data/2010/sf1?get=P0010001,P0040003&for=zip+code+tabulation+area:*&in=state:40'
response = requests.get(url).json()
#print (response)
census_variable_p0010001 = []
census_variable_p0040003 = []
zip_code = []
counter = 0
for data in response:
    if (counter > 0):
        census_variable_p0010001.append(data[0])
        census_variable_p0040003.append(data[1])
        zip_code.append(data[3])
    counter = counter + 1

zip_codes_for_state = {"Census Code" : census_variable_p0010001,
                      "Census Code (P0040003)" : census_variable_p0040003,
                      "Zip Code" : zip_code}
zip_codes_for_state_df = pd.DataFrame(zip_codes_for_state)
zip_codes_for_state_df.head()

,Census Code,Census Code (P0040003),Zip Code
0,146,64,67950
1,1062,45,73002
2,20617,1417,73003
3,857,29,73004
4,9541,1013,73005


In [63]:
crime_data = pd.read_csv('./Resources/crime-report.csv', encoding="utf-8")
crime_data_ok = crime_data[crime_data['state_abbr'] == 'OK']
crime_data_processed = pd.DataFrame()
crime_data_processed['Agency Name'] = crime_data_ok['agency_name']
crime_data_processed['County'] = crime_data_ok['primary_county']
crime_data_processed['FIPS'] = crime_data_ok['primary_county_fips']
clean_df = crime_data_processed.dropna()
grouped_county_crime = clean_df.groupby(['County'])
#print(grouped_county_crime.count())

In [91]:
highschool_dropout_data = pd.read_csv('./Resources/EducationReport-hd.csv', encoding="utf-8")
hs_data_processed = pd.DataFrame()
#highschool_dropout_data.head()
hs_data_processed['County'] = highschool_dropout_data['Unnamed: 1']
hs_data_processed['FIPS'] = highschool_dropout_data['Unnamed: 0']
hs_data_processed['Dropout Percentage'] = highschool_dropout_data['Unnamed: 8']

clean_hs_df = hs_data_processed.dropna()
temp_df = clean_hs_df[clean_hs_df['County'] != 'Name']
temp_df2 = temp_df[temp_df['County'] != 'Oklahoma']
print(temp_df2)

                      County  \
0                        NaN   
3          Adair, OK           
4        Alfalfa, OK           
5          Atoka, OK           
6         Beaver, OK           
7        Beckham, OK           
8         Blaine, OK           
9          Bryan, OK           
10         Caddo, OK           
11      Canadian, OK           
12        Carter, OK           
13      Cherokee, OK           
14       Choctaw, OK           
15      Cimarron, OK           
16     Cleveland, OK           
17          Coal, OK           
18      Comanche, OK           
19        Cotton, OK           
20         Craig, OK           
21         Creek, OK           
22        Custer, OK           
23      Delaware, OK           
24         Dewey, OK           
25         Ellis, OK           
26      Garfield, OK           
27        Garvin, OK           
28         Grady, OK           
29         Grant, OK           
30         Greer, OK           
31        Harmon, OK           
..      